# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [34]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [35]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[3065, 3066, 3067]


In [36]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "id": 3065,
    "name": "Melissa",
    "account": {
      "id": 3247,
      "number": "0090-1",
      "agency": "0000-2",
      "balance": 1000.0,
      "limit": 5000.0
    },
    "card": {
      "id": 2968,
      "number": "**** **** **** 0900",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3066,
    "name": "Marcos",
    "account": {
      "id": 3248,
      "number": "0090-2",
      "agency": "0000-2",
      "balance": 5000.0,
      "limit": 10000.0
    },
    "card": {
      "id": 2969,
      "number": "**** **** **** 0901",
      "limit": 6000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 3067,
    "name": "Jo\u00e3o",
    "account": {
      "id": 3249,
      "number": "0090-3",
      "agency": "0000-2",
      "balance": 300.0,
      "limit": 1000.0
    },
    "card": {
      "id": 2970,
      "number": "**** **** **** 0903",
      "limit": 2000.0
    },
    "features": [],
    "news": []
  }
]


## **T**ransform

Utilize a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [37]:
!pip install openai

In [38]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = ''

In [39]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em investimentos."
      },
      {
          "role": "user",
          "content": f"Faça uma breve apresentação do assunto (100 palavras) e cite uma lista de 5 itens resumida para alguém com R$ {user['account']['balance']} com nomes de empresas para comprar ações de investimento na data atual (citar preço da ação, organizar por risco atual)"

      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

Os investimentos são uma forma de aplicar o dinheiro visando obter retorno financeiro. Existem diversas opções de investimentos, mas um dos mais populares é a compra de ações de empresas negociadas na bolsa de valores. Ao adquirir ações, você se torna um acionista da empresa e pode se beneficiar de dividendos e da valorização das ações ao longo do tempo.

Para alguém com R$ 1000, é importante lembrar que a diversificação é fundamental. Dessa forma, é recomendado investir em diferentes empresas de setores variados. A seguir, uma lista resumida com 5 sugestões de empresas para considerar, organizadas pelo nível atual de risco:

1. Empresa A - Preço da ação: R$ 50 - Setor XPTO
2. Empresa B - Preço da ação: R$ 30 - Setor ABC
3. Empresa C - Preço da ação: R$ 20 - Setor DEF
4. Empresa D - Preço da ação: R$ 40 - Setor GHI
5. Empresa E - Preço da ação: R$ 60 - Setor JKL

Lembrando que essa lista é apenas uma sugestão baseada nos preços atuais das ações e setores de mercado. Antes de investir, 

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [40]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

User Melissa updated? False!
User Marcos updated? False!
User João updated? False!
